# Better aligned Wiki dataset : WikNet
http://ssli.ee.washington.edu/tial/projects/simplification/

In [3]:
import os.path
import os
import tensorflow as tf
import keras
import nltk
nltk.download('all')
from nltk import word_tokenize
from nltk.tokenize.moses import MosesTokenizer
from nltk.tokenize.moses import MosesDetokenizer
import pandas as pd
import numpy as np
import re
import codecs
import sys
import operator
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, './subword_nmt')
from subword_nmt import learn_bpe
from subword_nmt import apply_bpe
sys.path.insert(0, './nmt/nmt')

# sys.path.insert(0, '/Users/tengi/insight-project/lib/pwkp_dataset_parser')
# from pwkp_parser import parse_pwkp_file
# download the princeton wordnet, once run it once
# nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package ce

[nltk_data]    |   Unzipping corpora/senseval.zip.
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/sentiwordnet.zip.
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/shakespeare.zip.
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/sinica_treebank.zip.
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/smultron.zip.
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /home/paperspace/nltk_data...
[nltk_data]    |   Unzipping corpora/state_un

In [ ]:
#download the scores
!wget -nv http://ssli.ee.washington.edu/tial/projects/simplification/WikNet_word_pairs.txt -P ./data/WikNet/
#download annotations
!wget -nv http://ssli.ee.washington.edu/tial/projects/simplification/annotations.txt -P ./data/WikNet/
#download good alignment
!wget -nv http://ssli.ee.washington.edu/tial/projects/simplification/aligned-good-0.67.txt -P ./data/WikNet/
#download partial alignment
!wget -nv http://ssli.ee.washington.edu/tial/projects/simplification/aligned-good_partial-0.53.txt -P ./data/WikNet/
#downad uncategorized
!wget -nv http://ssli.ee.washington.edu/tial/projects/simplification/aligned-remaining-0.45.txt -P ./data/WikNet/

In [ ]:
wiknet_good_file = 'data/WikNet/aligned-good-0.67.txt'
wiknet_partial_file = 'data/WikNet/aligned-good_partial-0.53.txt'
wiknet_good_prefix = 'data/WikNet/good'
wiknet_partial_prefix = 'data/WikNet/partial'

wiknet_good_partial_prefix = 'data/WikNet/good_partial'


translations = []
# generate complex-simple paris in translations
def parse_wiknet_file(src_file):
    translations = []
    scores = []
    with open(src_file) as wiknet_file:
            for line in wiknet_file:
                triplet = line.split("\t")
                translations.append([triplet[0].strip(), triplet[1].strip()])
                scores.append(triplet[0].strip())               

    return (translations, scores);
def split_train_val_test(translations, src_prefix):
    np.random.shuffle(translations)
    complex_tok_str = [pair[0] for pair in translations]
    simple_tok_str = [pair[1] for pair in translations]
    
    corpus_file = src_prefix+'.all.tok'
    
    complex_file = src_prefix+'.tok.complex'
    simple_file = src_prefix+'.tok.simple'
    
    complex_train_file = src_prefix+'.train.tok.complex'
    simple_train_file = src_prefix+'.train.tok.simple'

    complex_val_file = src_prefix+'.val.tok.complex'
    simple_val_file = src_prefix+'.val.tok.simple'

    complex_test_file = src_prefix+'.test.tok.complex'
    simple_test_file = src_prefix+'.test.tok.simple'

    val_size = 300
    test_size = 300

    with open(corpus_file, 'w') as f:
        for i in range(len(complex_tok_str)):
            f.write(complex_tok_str[i] + '\n')
            f.write(simple_tok_str[i] + '\n\n')


    with open(complex_file, 'w') as f:
        for sent in complex_tok_str:
            f.write(sent + '\n')

    with open(simple_file, 'w') as f:
        for sent in simple_tok_str:
            f.write(sent + '\n')

    with open(complex_train_file, 'w') as f:
        for sent in complex_tok_str[:-(val_size+test_size)]:
            f.write(sent + '\n')

    with open(simple_train_file, 'w') as f:
        for sent in simple_tok_str[:-(val_size+test_size)]:
            f.write(sent + '\n')

    with open(complex_val_file, 'w') as f:
        for sent in complex_tok_str[-(val_size+test_size):-test_size]:
            f.write(sent + '\n')

    with open(simple_val_file, 'w') as f:
        for sent in simple_tok_str[-(val_size+test_size):-test_size]:
            f.write(sent + '\n')

    with open(complex_test_file, 'w') as f:
        for sent in complex_tok_str[-test_size:]:
            f.write(sent + '\n')

    with open(simple_test_file, 'w') as f:
        for sent in simple_tok_str[-test_size:]:
            f.write(sent + '\n')


good_translations, _= parse_wiknet_file(wiknet_good_file)
split_train_val_test(good_translations, wiknet_good_prefix)

partial_translations, _= parse_wiknet_file(wiknet_partial_file)
split_train_val_test(partial_translations, wiknet_partial_prefix)

split_train_val_test(good_translations+partial_translations, wiknet_good_partial_prefix)

In [ ]:
# Learn a byte-pair-encoding (bpe) vocabulary using 10,000 merge operations
!./subword_nmt/learn_bpe.py -s 10000 < data/WikNet/good_partial.all.tok > data/WikNet/codes.bpe

# Apply the vocabulary to the training file
!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.all.tok > data/WikNet/good_partial.tok.bpe
!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.tok.complex > data/WikNet/good_partial.tok.bpe.complex
!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.tok.simple > data/WikNet/good_partial.tok.bpe.simple

!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.train.tok.complex > data/WikNet/good_partial.train.tok.bpe.complex

!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.train.tok.simple > data/WikNet/good_partial.train.tok.bpe.simple

!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.val.tok.complex > data/WikNet/good_partial.val.tok.bpe.complex

!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.val.tok.simple > data/WikNet/good_partial.val.tok.bpe.simple

!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.test.tok.complex > data/WikNet/good_partial.test.tok.bpe.complex

!./subword_nmt/apply_bpe.py -c data/WikNet/codes.bpe < data/WikNet/good_partial.test.tok.simple > data/WikNet/good_partial.test.tok.bpe.simple

!cat data/WikNet/good_partial.tok.bpe | ./subword_nmt/get_vocab.py > data/WikNet/good_partial.vocab.tok.bpe
!cat data/WikNet/good_partial.tok.bpe.complex | ./subword_nmt/get_vocab.py > data/WikNet/vocab.tok.bpe.complex
!cat data/WikNet/good_partial.tok.bpe.simple | ./subword_nmt/get_vocab.py > data/WikNet/vocab.tok.bpe.simple

In [ ]:
tokenizer = MosesTokenizer()
good_partial_translations = good_translations+partial_translations

complex_sentences = [pair[0] for pair in good_partial_translations]
simple_sentences = [pair[1] for pair in good_partial_translations]

complex_tok_str = [tokenizer.tokenize(sent, return_str = True) for sent in complex_sentences]
simple_tok_str = [tokenizer.tokenize(sent, return_str = True) for sent in simple_sentences] 

complex_tok_words = [word for sent in complex_tok_str for word in sent.split()]
simple_tok_words = [word for sent in simple_tok_str for word in sent.split()]

In [ ]:
complex_fdist = nltk.FreqDist(complex_tok_words)
simple_fdist = nltk.FreqDist(simple_tok_words)

complex_chars = sum([len(word) for word in complex_fdist.keys()])
simple_chars = sum([len(word) for word in simple_fdist.keys()])

df = pd.DataFrame({'name': ['complex', 'simple'],
                   '#tokens/sentence': [complex_fdist.N()/len(complex_sentences), simple_fdist.N()/len(simple_sentences)],
                   '#chars/tokens': [complex_chars/complex_fdist.B(), simple_chars/simple_fdist.B()],
                   '#unique tokens': [complex_fdist.B(), simple_fdist.B()]})
df.set_index('name')

In [ ]:
fdist = nltk.FreqDist(complex_tok_words+simple_tok_words)
print("Number of tokens before subword segmentation: {}".format(fdist.B()))

text_file = open(r'data/WikNet/good_partial.tok.bpe',"r")
p = text_file.read()
words = word_tokenize(p)
fdist = nltk.FreqDist(words)
print("Number of tokens after subword segmentation: {}".format(fdist.B()))
